# Импорт библиотек

In [17]:
import pandas as pd
from scipy.stats import chi2_contingency

# Изучение данных

In [18]:
df = pd.read_csv('/kaggle/input/ad-ab-testing/AdSmartABdata - AdSmartABdata.csv')

In [19]:
df.head(3)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1


In [20]:
df = df.drop(columns=['auction_id'])

In [21]:
def df_stats(data):
    print("***РАЗМЕР***")
    print("Строк: {}".format(data.shape[0]))
    print("Столбцов: {}".format(data.shape[1]))
    print("***ТИПЫ ДАННЫХ***")
    print(data.dtypes)
    print("***ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ***")
    print(data.isnull().sum())
    print("***ЧИСЛО ДУБЛИКАТОВ***")
    print("Количество дубликатов: {}".format(data.duplicated().sum()))
df_stats(df)

***РАЗМЕР***
Строк: 8077
Столбцов: 8
***ТИПЫ ДАННЫХ***
experiment     object
date           object
hour            int64
device_make    object
platform_os     int64
browser        object
yes             int64
no              int64
dtype: object
***ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ***
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64
***ЧИСЛО ДУБЛИКАТОВ***
Количество дубликатов: 4435


In [22]:
df['no'].unique()

array([0, 1])

In [23]:
df['yes'].unique()

array([0, 1])

Данные чистые

In [24]:
df.describe().T # Делаем сводную статистику, транспонируем для лучшей читаемости

,count,mean,std,min,25%,50%,75%,max
hour,8077.0,11.615080,5.734879,0.0,7.0,13.0,15.0,23.0
platform_os,8077.0,5.947134,0.224333,5.0,6.0,6.0,6.0,7.0
yes,8077.0,0.070818,0.256537,0.0,0.0,0.0,0.0,1.0
no,8077.0,0.083075,0.276013,0.0,0.0,0.0,0.0,1.0


In [25]:
print("Experiment :", df['experiment'].unique())
print("platform_os :", df['platform_os'].unique())
print("Yes :", df['yes'].unique())
print("No :", df['no'].unique())

Experiment : ['exposed' 'control']
platform_os : [6 5 7]
Yes : [0 1]
No : [0 1]


In [26]:
yes_val = df[df['yes']==1]
yes_val.shape

(572, 8)

In [27]:
no_val = df[df['no']==1]
no_val.shape

(671, 8)

In [28]:
df.shape

(8077, 8)

# Предварительная обработка данных

In [29]:
# Много строк без ответов
No_response = df.shape[0] - (yes_val.shape[0]+no_val.shape[0])
print(No_response)

6834


In [30]:
df1 = df[(df['yes'] == 1) | (df['no'] == 1)]

In [31]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1243 entries, 2 to 8071
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   experiment   1243 non-null   object
 1   date         1243 non-null   object
 2   hour         1243 non-null   int64 
 3   device_make  1243 non-null   object
 4   platform_os  1243 non-null   int64 
 5   browser      1243 non-null   object
 6   yes          1243 non-null   int64 
 7   no           1243 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 87.4+ KB


In [32]:
rate = df1.groupby('experiment')[['yes','no']].sum()

In [33]:
rate['conversion_rate'] = round(rate['yes'] / (rate['yes'] + rate['no']), 4)

In [34]:
rate

,yes,no,conversion_rate
experiment,,,
control,264,322,0.4505
exposed,308,349,0.4688


# A/B Тест

In [35]:
contingency_table = [
    [rate.loc['control', 'yes'], rate.loc['control', 'no']],
    [rate.loc['exposed', 'yes'], rate.loc['exposed', 'no']]
]

# Хи-квадрат
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print("Статистика Хи-квадрат:", round(chi2, 4))
print("P-значение:", round(p_value, 4))

Статистика Хи-квадрат: 0.3465
P-значение: 0.5561


# Результат

#### Р-значение, равное 0,5561, намного выше обычного уровня значимости 0,05. Это означает, что мы не можем отвергнуть нулевую гипотезу, предполагающую, что разница в коэффициентах конверсии между контрольной и подвергшейся воздействию группами не является статистически значимой